In [ ]:
import numpy as np
#import cv2
import PIL
import os
import cv2
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers,datasets,models
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras import Sequential,layers

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir=tf.keras.utils.get_file('flower_photos',origin=dataset_url,cache_dir='.',untar=True)


In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)

In [ ]:
data_dir

In [ ]:
len(list(data_dir.glob('*/*.jpg')))

In [ ]:
roses = list(data_dir.glob('roses/*'))

In [ ]:
PIL.Image.open(str(roses[2]))

In [ ]:
flower_images_dict = {
    'roses':list(data_dir.glob('roses/*')),
    'daisy':list(data_dir.glob('daisy/*')),
    'dandelion':list(data_dir.glob('dandelion/*')),
    'sunflowers':list(data_dir.glob('sunflowers/*')),
    'tulips':list(data_dir.glob('tulips/*')),
}

In [ ]:
flower_label_dict = {
    'roses':0,
    'daisy':1,
    'dandelion':2,
    'sunflowers':3,
    'tulips':4,
}

In [ ]:
img = cv2.imread(str(flower_images_dict['roses'][0]))
img.shape

In [ ]:
cv2.resize(img,(180,180)).shape

In [ ]:
X,y = [], []
for flower_name,images in flower_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
       # print(img.shape)
        resized_img = cv2.resize(img,(180,180))
        #print(resized_img.shape)
        X.append(resized_img)
        y.append(flower_label_dict[flower_name])

In [ ]:
X = np.array(X)
y = np.array(y)
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)

In [ ]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255
X_train.shape

In [66]:
model = Sequential([
    #cnn
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    #Dense
    layers.Flatten(input_shape=()),
    layers.Dense(128,activation='relu'),
    layers.Dense(5)
])
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X_train_scaled,y_train,epochs=5)

Epoch 1/5
86/86 [==============================] - 47s 526ms/step - loss: 1.4124 - accuracy: 0.3797
Epoch 2/5
86/86 [==============================] - 43s 505ms/step - loss: 1.0776 - accuracy: 0.5560
Epoch 3/5
86/86 [==============================] - 44s 506ms/step - loss: 0.9225 - accuracy: 0.6439
Epoch 4/5
86/86 [==============================] - 43s 500ms/step - loss: 0.7684 - accuracy: 0.7039
Epoch 5/5
86/86 [==============================] - 42s 491ms/step - loss: 0.5525 - accuracy: 0.7980


In [67]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 3s 83ms/step - loss: 0.9785 - accuracy: 0.6525


[0.9784730076789856, 0.6525054574012756]

In [69]:
predictions=model.predict(X_test_scaled)

29/29 [==============================] - 2s 78ms/step


In [72]:
yans=[]
for i in predictions:
    yans.append(np.argmax(tf.nn.softmax(i)))
yans[0]

1

In [73]:
y_test[0]

1

In [74]:
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomZoom(0.3),
])

In [75]:
model = Sequential([
    #cnn
    data_augmentation,
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    #Dense
    layers.Flatten(input_shape=()),
    layers.Dense(128,activation='relu'),
    layers.Dense(5)
])
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X_train_scaled,y_train,epochs=5)

Epoch 1/5
86/86 [==============================] - 50s 551ms/step - loss: 1.3457 - accuracy: 0.4201
Epoch 2/5
86/86 [==============================] - 44s 511ms/step - loss: 1.0749 - accuracy: 0.5669
Epoch 3/5
86/86 [==============================] - 45s 524ms/step - loss: 0.9462 - accuracy: 0.6384
Epoch 4/5
86/86 [==============================] - 44s 507ms/step - loss: 0.8637 - accuracy: 0.6632
Epoch 5/5
86/86 [==============================] - 45s 520ms/step - loss: 0.7847 - accuracy: 0.6926


In [76]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 3s 82ms/step - loss: 0.9388 - accuracy: 0.6492


[0.9388249516487122, 0.6492374539375305]